- CARTESI/O CONFIG FILE:
========================
Here we are going to compile a file to specify solver and control problem (Inverse Kinematics) for the CartesI/O framework.
The data specified will be used to instantiate ```Tasks```, ```Constraints``` and ```Solver``` from OpenSoT as well as ```Interactive Markers``` and ROS services and topics.

Solver Options:
----------------------
The first thing to include are related to the back-end used by the front-end which is ```iHQP/LP```:
```
solver_options:
    regularization: 1e-6
    back_end: "osqp"
```
```regularization``` is a constant used for the solver regularization term, ```back_end``` specify the particular solver used, in our case ```osqp``` (you can try to use ```qpoases``` as well).

IK Problem Specification:
------------------------------------
First we decide which tasks we want to solve and which priorities, for example:
```
stack:
    - ["LFoot","RFoot","Head"]
    - ["CoMXY","WaistYaw"]
    - ["LArm","RArm"]
    - ["BP"]
```
here we have at the first level: ```LFoot```,```RFoot``` and ```Head``` which are all at the same first priority. The first two are full 6DoFs Cartesian tasks while the latter in a postural task to move the joints of the head. At second priority there are ```CoMXY``` and ```WaistYaw```; in this case we are interested to control 2 Cartesian directions for the CoM task and only the Yaw rotation for the Waist. Third priority we have both arms, 6DoFs Cartesian tasks and finally, last priority a postural. 
For what concern constraints, we want to consider joint limits and joint velocity limits, this can be done as:
```
constraints: ["JointLimits", "VelocityLimits"]
```
in particular, these constraints will acts to all the levels of the stack.